# CLgen Sampling
-----
Dec 2016.

In [4]:
# preamble
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from labm8 import fs
from labm8 import viz

import clgen
from clgen import corpus

## Experimental methodology

Experimental platforms: cc[123]. NVIDIA GTX 1080.

* Train on GitHub corpus.
* Generate 1000 samples.
* Reject bad samples.

Commands:

```
$ export CUDA_VISIBLE_DEVICES=0
$ export MODEL=model-128x3x50
$ cd ~/phd/lab/clgen/inference
$ python ./inference.py $MODEL.json 2>&1 | tee inference-$MODEL.log
```

In [5]:
c = corpus.Corpus.from_json({"path": "~/data/github"})
print("CLgen:      ", clgen.version())
print("Corpus size:", c.size)
print("Vocab size: ", c.vocab_size)

CLgen:       0.2.1
Corpus size: 20271852
Vocab size:  92


In [6]:
p, num_prototypes = corpus.most_common_prototypes(c, 20)
for row in p:
    ratio, prototype = row
    print(ratio * 100, '\t', prototype)

print("# prototypes", num_prototypes)

AttributeError: module 'clgen.corpus' has no attribute 'most_common_prototypes'

## Experimental results

In [3]:
def parse_file(path):
    """ return model size, num layers, and epochs """
    import re
    m = re.search(r"([0-9]+)x([0-9]+)x([0-9]+)", path)
    return m.group(1), m.group(2), m.group(3) 

data_files = ["benchmark/" + x for x in fs.ls("benchmark")
              if x.startswith("benchmark-") and x.endswith(".json")]
data = dict((parse_file(x), clgen.load_json_file(x)) for x in data_files)
print("read {} data files".format(len(data)))

read 6 data files


## Evaluation

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
from labm8 import viz
plt.style.use(["seaborn-white", "seaborn-paper"])

def label(path):
    return '-'.join([str(x) for x in path])

# sort key values
sorted_keys = sorted(data.keys(), key=lambda x: [int(y) for y in x])